# Sensitivity Analysis in Minutes

- **Goal:** Run a one-factor sensitivity sweep on the Midazolam reference model using the in-memory adapter.
- **Data:** `reference/models/standard/midazolam_adult.pkml`.
- **Targets:** Completion < 10 minutes on 8 vCPU; PK metric deltas within ±1%.


## Steps

1. Install dependencies with `pip install -e '.[dev]'`.
2. Execute the cell below to spin up an in-memory adapter, run the sensitivity workflow, and collect PK metrics.
3. Compare the reported deltas against the 1% tolerance noted in the performance plan.


In [ ]:
from pathlib import Path

from mcp import session_registry
from mcp_bridge.adapter.interface import AdapterConfig
from mcp_bridge.adapter.mock import InMemoryAdapter
from mcp_bridge.agent.sensitivity import (
    SensitivityConfig,
    SensitivityParameterSpec,
    run_sensitivity_analysis,
)
from mcp_bridge.services.job_service import JobService

fixture = Path("reference/models/standard/midazolam_adult.pkml").resolve()
adapter = InMemoryAdapter(AdapterConfig(model_search_paths=[str(fixture.parent)]))
adapter.init()
job_service = JobService(max_workers=2, default_timeout=120.0, max_retries=0)

try:
    config = SensitivityConfig(
        model_path=fixture,
        base_simulation_id="usecase-sens",
        parameters=[
            SensitivityParameterSpec(
                path="Organism|Weight",
                deltas=[-0.1, 0.1],
                unit="kg",
            )
        ],
    )
    report = run_sensitivity_analysis(adapter, job_service, config)
    print(report.model_dump_json(indent=2))
finally:
    job_service.shutdown()
    adapter.shutdown()
    session_registry.clear()


## Reference Metrics

The parity suite establishes the baseline PK metrics for Midazolam as `cmax=1.000`,
`tmax=1.000`, `auc=0.500`. The sensitivity report should show percentage deltas within
±1% for both the -10% and +10% parameter adjustments.
